## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). 

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that. 

Fill in your **NAME** and **AEM** below:

In [1]:
NAME = "Αλέξανδρος Τσίγγος"
AEM = "3690"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [2]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

In [3]:
# Always run this cell
# Στο κελί αυτό συμπεριλαμβάνονται όλα τα imports που θα χρειαστούν στο notebook

from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [24]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

('test_set_noclass.csv', <http.client.HTTPMessage at 0x27a65194d48>)

In [4]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **10-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation. Otherwise, you can use [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses three **simple** estimators/classifiers. Test both soft and hard voting and choose the best one. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   Probabilistic Models (Naive Bayes)
*   KNN Models

In [4]:
# BEGIN CODE HERE

cls1 = DecisionTreeClassifier(random_state = RANDOM_STATE) # Classifier #1
cls2 = LogisticRegression(random_state = RANDOM_STATE, n_jobs=-1) # Classifier #2
cls3 = KNeighborsClassifier(n_neighbors=3, n_jobs=-1) # Classifier #1
soft_vcls = VotingClassifier(estimators=[('DTC', cls1), ('LR', cls2), ('KNN', cls3)], voting='soft') # Voting Classifier
hard_vcls = VotingClassifier(estimators=[('DTC', cls1), ('LR', cls2), ('KNN', cls3)], voting='hard') # Voting Classifier

svlcs_scores = cross_validate(soft_vcls, X, y, scoring = ['f1_weighted', 'balanced_accuracy'], cv=10, n_jobs=-1)
s_avg_fmeasure = sum(svlcs_scores['test_f1_weighted'])/len(svlcs_scores['test_f1_weighted']) # The average f-measure
s_avg_accuracy = sum(svlcs_scores['test_balanced_accuracy'])/len(svlcs_scores['test_balanced_accuracy']) # The average accuracy

hvlcs_scores = cross_validate(hard_vcls, X, y, scoring = ['f1_weighted', 'balanced_accuracy'], cv=10, n_jobs=-1)
h_avg_fmeasure = sum(hvlcs_scores['test_f1_weighted'])/len(hvlcs_scores['test_f1_weighted']) # The average f-measure
h_avg_accuracy = sum(hvlcs_scores['test_balanced_accuracy'])/len(hvlcs_scores['test_balanced_accuracy']) # The average accuracy
#END CODE HERE

KeyboardInterrupt: 

In [31]:
print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('DTC', DecisionTreeClassifier(random_state=42)),
                             ('LR',
                              LogisticRegression(n_jobs=-1, random_state=42)),
                             ('KNN',
                              KNeighborsClassifier(n_jobs=-1, n_neighbors=3))],
                 voting='soft')
F1 Weighted-Score: 0.8232 & Balanced Accuracy: 0.816


You should achive above 82% (Soft Voting Classifier)

In [32]:
print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('DTC', DecisionTreeClassifier(random_state=42)),
                             ('LR',
                              LogisticRegression(n_jobs=-1, random_state=42)),
                             ('KNN',
                              KNeighborsClassifier(n_jobs=-1, n_neighbors=3))])
F1 Weighted-Score: 0.8261 & Balanced Accuracy: 0.8191


You should achieve above 80% in both! (Hard Voting Classifier)

### 1.2 Stacking ###
Create a stacking classifier which uses two more complex estimators. Try different simple classifiers (like the ones mentioned before) for the combination of the initial estimators. Report your results in the following cell.

Consider as complex estimators the following:

*   Random Forest
*   SVM
*   Gradient Boosting
*   MLP




In [7]:

# BEGIN CODE HERE
cls1 = MLPClassifier(random_state=RANDOM_STATE, max_iter=100) # Classifier #1
cls2 = SVC(gamma='auto',random_state=RANDOM_STATE) # Classifier #2
cls3 = "" # Classifier #3 (Optional)
scls = StackingClassifier(estimators=[('SVC',cls1),('MLP',cls2)],final_estimator=LogisticRegression(random_state=RANDOM_STATE,n_jobs=-1),n_jobs=-1) # Stacking Classifier
scores = cross_validate(scls,X,y,scoring = ['f1_weighted', 'balanced_accuracy'], cv=10, n_jobs=-1)
avg_fmeasure = sum(scores['test_f1_weighted'])/len(scores['test_f1_weighted']) # The average f-measure
avg_accuracy = sum(scores['test_balanced_accuracy'])/len(scores['test_balanced_accuracy']) # The average accuracy
# Παραθέτω το output, γιατί παίρνει λίγο χρόνο για να τρέξει
# F1 Weighted Score: 0.865 & Balanced Accuracy: 0.8589
#END CODE HERE

In [8]:
print("Classifier:")
print(scls)
print("F1 Weighted Score: {} & Balanced Accuracy: {}".format(round(avg_fmeasure,4), round(avg_accuracy,4)))

Classifier:
StackingClassifier(estimators=[('SVC',
                                MLPClassifier(max_iter=100, random_state=42)),
                               ('MLP', SVC(gamma='auto', random_state=42))],
                   final_estimator=LogisticRegression(n_jobs=-1,
                                                      random_state=42),
                   n_jobs=-1)
F1 Weighted Score: 0.865 & Balanced Accuracy: 0.8589


You should achieve above 85% in both

## 2.0 Randomization ##

**2.1** You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements.  

In [12]:
# BEGIN CODE HERE
ens1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=RANDOM_STATE),n_estimators=200, n_jobs=-1,random_state=RANDOM_STATE)
ens2 = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE,n_jobs=-1)
ens3 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=6,min_samples_leaf=5),n_estimators=100,random_state=RANDOM_STATE)
tree = DecisionTreeClassifier(random_state=RANDOM_STATE)

ens1_scores = cross_validate(ens1,X,y,scoring = ['f1_weighted', 'balanced_accuracy'], cv=3, n_jobs=-1)
ens2_scores = cross_validate(ens2,X,y,scoring = ['f1_weighted', 'balanced_accuracy'], cv=3, n_jobs=-1)
ens3_scores = cross_validate(ens3,X,y,scoring = ['f1_weighted', 'balanced_accuracy'], cv=3, n_jobs=-1)
tree_scores = cross_validate(tree,X,y,scoring = ['f1_weighted', 'balanced_accuracy'], cv=3, n_jobs=-1)

f_measures = dict(Bagging = sum(ens1_scores['test_f1_weighted'])/len(ens1_scores['test_f1_weighted']), RandomForest = sum(ens2_scores['test_f1_weighted'])/len(ens2_scores['test_f1_weighted']),AdaBoost = sum(ens3_scores['test_f1_weighted'])/len(ens3_scores['test_f1_weighted']), Simple_DT = sum(tree_scores['test_f1_weighted'])/len(tree_scores['test_f1_weighted']))
accuracies = dict(Bagging = sum(ens1_scores['test_balanced_accuracy'])/len(ens1_scores['test_balanced_accuracy']), RandomForest = sum(ens2_scores['test_balanced_accuracy'])/len(ens2_scores['test_balanced_accuracy']),AdaBoost = sum(ens3_scores['test_balanced_accuracy'])/len(ens3_scores['test_balanced_accuracy']), Simple_DT = sum(tree_scores['test_balanced_accuracy'])/len(tree_scores['test_balanced_accuracy']))
# Example f_measures = {'Simple Decision': 0.8551, 'Ensemble with random ...': 0.92, ...}

#END CODE HERE

In [13]:
print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier:{} -  F1 Weighted:{}".format(name,round(score,4)))
for name,score in accuracies.items():
    print("Classifier:{} -  BalancedAccuracy:{}".format(name,round(score,4)))

BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=42),
                  n_estimators=200, n_jobs=-1, random_state=42)
RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0)
AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6,
                                                         min_samples_leaf=5,
                                                         random_state=42),
                   n_estimators=100, random_state=42)
DecisionTreeClassifier(random_state=42)
Classifier:Bagging -  F1 Weighted:0.8056
Classifier:RandomForest -  F1 Weighted:0.7996
Classifier:AdaBoost -  F1 Weighted:0.797
Classifier:Simple_DT -  F1 Weighted:0.6973
Classifier:Bagging -  BalancedAccuracy:0.7926
Classifier:RandomForest -  BalancedAccuracy:0.784
Classifier:AdaBoost -  BalancedAccuracy:0.7841
Classifier:Simple_DT -  BalancedAccuracy:0.6896


**2.2** Describe your classifiers and your results.

<ul>
    <li>Ο πρώτος classifier που χρησιμοποιήσαμε παράχθηκε με τη μέθοδο του Bagging όπου η παραγωγή των μοντέλων του ensemble επιτυγχάνεται μέσω της εφαρμογής του ίδιου αλγορίθμου (εδώ χρησιμοποιήσαμε Decision Tree Classifier) σε <b>διαφορετικές παραλλαγές του συνόλου εκπαίδευσης</b>, οι οποίες προκύπτουν χρησιμοποιώντας δειγματοληψία με επανατοποθέτηση (η διαδικασία αυτή έγινε 200 φορές λόγω του n_estimators=200). Τα αποτελέσματα και με τις δυο μετρικές είναι κοντά στο 0.8</li>
    <li>Ο δεύτερος classifier χρησιμοποιεί την μέθοδο του Random Forest, όπου σε κάθε κόμβο επιλέγεται το καλύτερο ζεύγος μεταβλητής εισόδου και τιμής μεταξύ όχι όλων των διαθέσιμων αλλά ενός τυχαία επιλεγμένου υποσυνόλου των features. Να τονίσουμε ότι χρησιμοποίηθηκαν, όπως και παραπάνω, 200 estimators. Τα αποτελέσματα και με τις δυο μετρικές είναι και πάλι κοντά στο 0.8</li>
    <li>Ο τρίτος classifier ακολουθεί την τεχνική του boosting, και πιο συγκεκριμένα χρησιμοποιήσαμε την προσαρμοστική ενίσχυση (AdaBoost).Στην συγκεκριμένη περίπτωση χρησιμοποιήσαμε ως base estimator ένα Decision Tree(max_depth=6,min_samples_leaf=5). Να τονίσουμε ότι χρησιμοποίηθηκαν 100 estimators, που σημαίνει ότι σε κάθε μία από αυτές τις 100 επαναλήψεις προσπαθούμε να κάνουμε το μοντέλο να διορθώσει τα λάθη των προηγούμενων μοντέλων, δίνοντας μεγαλύτερη προσοχή στα παραδείγματα εκπαίδευσης όπου τα προηγούμενα μοντέλα εμφανίζουν την μργαλύτερη απόκλιση από την μεταβλητή στόχο. Το απότελεσμα του F1 σε αυτή την περίπτωση είναι 0.79 και 0.78 στο balanced accuracy.</li>
    <li>Αναφορικά με τον τρόπο σύγκρισης των 3 ensembles και του απλού Decision Tree Classifier, να τονίσουμε ότι κάναμε cross validation σε κάθε έναν από τους classifiers και στο τέλος πήραμε των Μ.Ο. των scores του καθενός έτσι ώστε να μπορέσουμε να συγκρίνουμε τα results των διαφορετικών classifiers</li>
    <li>Συγκεντρωτικά, παρατηρούμε πώς και τα 3 ensembles πετυχαίνουν περίπου 0.8 και στις 2 μετρικές, ενώ ο απλός Decision Classifier πετυχαίνει μικρότερο score γύρω στο 0.7 και στις 2 μετρικές, το οποίο είναι και αναμενόμενο μιας και είδαμε στην θεωρία τους λόγους για τους οποίους τα ensembles ξεπερενούν (outperform) τα single models</li>
</ul>

**2.3** Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

<p>Ναι, υπάρχει λύση στο συγκεκριμένο πρόβλημα και αύτη είναι η παραλληλοποίηση της διαδικασίας του bagging. Πιο συγκεκριμένα, στο bagging τα μοντέλα του ensemble μπορούν να εκπαιδευτοούν ανεξάρτητα το ένα από το άλλο και συνεπώς θα μπορούσε να γίνει χρήση πολλαπλών υπολογιστικών πόρων για την επιτάχυνση των διαδικασιών εκπαίδευσης. Π.χ. κάθε ένα από τα threads της cpu θα μπορούσε να εκπαιδεύει και ένα μοντέλο του ensemble. Παραπάνω κάναμε χρήση αυτής της λύσης ώστε να μειώσουμε το trainning time του classifier δίνοντας ως παράμετρο στο ensemble το n_jobs=-1. Ωστόσο, μια τέτοια λύση <b>δεν</b> μπορεί να εφαρμοστεί στην περίπτωση ενός boosting classifier (όπως π.χ. του AdaBoost που χρησιμοποιήσαμε παραπάνω). Αυτό συμβαίνει επειδή τα διαφορετικά μοντέλα παράγονται <b>ακολουθιακά</b>,καθώς κάθε μοντέλο παίρνει είσοδο από το προηγούμενο μοντέλο και αυτή η διαδικασία δεν μπορεί να παραλληλοποιηθεί.</p>

## 3.0 Creating the best classifier ##

**3.1** In this part of the assignment you are asked to train the best possible ensemble! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code. Can you achieve a balanced accuracy over 83-84%?

In [5]:
# BEGIN CODE HERE
cls1 = MLPClassifier(random_state=RANDOM_STATE, max_iter=100,alpha=0.0001, hidden_layer_sizes=(100,),learning_rate='adaptive')
cls2 = AdaBoostClassifier(base_estimator=LogisticRegression(random_state=RANDOM_STATE,n_jobs=-1) ,n_estimators=10, random_state=RANDOM_STATE)
cls3 = SVC(gamma='auto',random_state=RANDOM_STATE)
best_cls = StackingClassifier(estimators=[('MLP',cls1),('AdaBoost',cls2),('SVC',cls3)],final_estimator=LogisticRegression(random_state=RANDOM_STATE,n_jobs=-1),n_jobs=-1)

scores = cross_validate(best_cls,X,y,scoring = ['f1_weighted', 'balanced_accuracy'], cv=10, n_jobs=-1)

best_fmeasure = sum(scores['test_f1_weighted'])/len(scores['test_f1_weighted'])
best_accuracy = sum(scores['test_balanced_accuracy'])/len(scores['test_balanced_accuracy'])
#END CODE HERE

In [6]:
print("Classifier:")
#print(best_cls)
print("F1 Weighted-Score:{} & Balanced Accuracy:{}".format(best_fmeasure, best_accuracy))

Classifier:
F1 Weighted-Score:0.8652637336589921 & Balanced Accuracy:0.8593066509423612


**3.2** Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code.

<ul>
    <li>Describe the process you followed to achieve this result
        <ol>
            <li>Στην προσπάθεια να δημιουργήσουμε τον "best classifier" θα χρησιμοποιήσουμε την τεχνική του stacking, με την οποία θα συνδυάσουμε 3 μοντέλα.Το μέτα-μοντέλο που θα χρησιμοποιηθεί στο stacking είναι ένας απλός Logistic Regression.</li>
            <li>1ο μοντέλο:
            Αρχικά, κάναμε ένα GridSearchCV για να βρούμε το καλύτερο set παραμέτρων για τον MLP Classifier, ο οποίος είναι και ο πρώτος base estimator που χρησιμοποιήσαμε.
            Παραθέτω εδώ το αποτέλεσμα του GridSearchCV: Best parameters found: {'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive'}</li>
            <li>2ο μοντέλο:
            Σε αυτήν την περίπτωση χρησιμοποιήσαμε τη μέθοδο του boosting, έχοντας ως base estimator έναν Logistic Regression. Γνωρίζουμε από τη θεωρία ότι όσους πιο πολλούς estimators χρησιμοποιούμε τόσο καλύτερα αποτελέσματα βλέπουμε, αλλά για να μην αυξηθεί επιπλέον το υπολογιστικό κόστος προτιμήσαμε να δώσουμε ως παράμετρο n_estimators=10.</li>
            <li>3ο μοντέλο:
            Σε αυτήν την περίπτωση προτιμήθηκε ως base estimator ένα SVC με παράμετρο gamma='auto'</li>
        </ol>
    </li>
    <li>How did you choose your classifier and your parameters and why
        <ol>
            <li>Δοκιμάσαμε 8 διαφορετικούς base estimators, από τους οποίους επιλέξαμε εκείνους τους τρεις που πέτυχαν τα καλύτερα αποτελέσματα MLP, AdaBoost, SVM) και τους δώσαμε ως είσοδο στον Stacking Classifier. Στην διαδικασία αυτή παρατηρήσαμε ότι το Random Forest και γενικά τα δενδρικά μοντέλα δεν ήταν ιδιαίτερα αποδοτικά σε αυτό το Data Set, ενώ αποφύγαμε τις τεχνικές του Bagging και του Rnadom Patches μιας και είναι ιδιαίτερα χρονοβόρες.</li>
            <li>Για να καταφέρει ένα ensemble να έχει μεγαλύτερη ορθότητα από τα επιμέρους μοντέλα, από τα οποία αποτελείται, θα πρέπει να περιλαμβάνει μοντέλα ικανοποιητικής ορθότητας και τα οποία να διαφέρουν μεταξύ τους ως προς τις περιπτώσεις στις οποίες σφάλλουν.</li>
            <li>Για αυτό ακριβώς τον λόγο επλέχθηκαν μοντέλα διαφορετικών οικογενειών (νευρωνικά, γραμμικά), όπου το κάθε ένα από αυτά τα τρία πετυχαίνει από μόνο του μια αρκετά ικανοποιητική ορθότητα, ενώ εξασφαλίζεται και το diversity. Ο συνδυασμός αυτών μέσω του stacking, το οποίο είναι ένας πιο intelligent τρόπος συγκερασμού μοντέλων, μας οδήγησε στο να πετύχουμε ακόμα καλύτερα αποτελέσματα.</li>
        </ol>
    </li>
    <li>Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code
        <ol>
            <li>F1 Weighted-Score: 0.8652637336589921</li>
            <li>Balanced Accuracy: 0.8593066509423612</li>
            <li>Παράκάτω έχουμε προσθέσει 2 cells, όπου στο πρώτο cell φαίνεται το Grid Search CV που κάναμε, ενώ στο δεύτερο cell φαίνεται ο πειραματισμός που κάναμε προκειμένου να καταλήξουμε στους καλύτερους base estimators και τα διάφορα results των base estimators που δοκιμάσαμε.</li>
            <li>results of classifiers you tried:
                <br>
                Gradient Boosting:<br>
                F1-Score: 0.8246217930362637
                Accuracy: 0.8151808296208805
                <br>
                SVC:<br>
                F1-Score: 0.8468680264642948
                Accuracy: 0.8392989348362883
                <br>
                MLP:<br>
                F1-Score: 0.851622807347304
                Accuracy: 0.8458722231494439
                <br>
                Bagging:<br>
                F1-Score: 0.8409584742455878
                Accuracy: 0.8358229234870953
                <br>
                Random Forest:<br>
                F1-Score: 0.8066134353605884
                Accuracy: 0.7929703046565072
                <br>
                Random Patches:<br>
                F1-Score: 0.8399049624791819
                Accuracy: 0.8333087262970047
                <br>
                AdaBoost:<br>
                F1-Score: 0.8526366007300631
                Accuracy: 0.8483864203395343
                <br>
                LogisticRegression:<br>
                F1-Score: 0.8329846976316241
                Accuracy: 0.8275549592807042
            </li>
        <ol>
    </li>
</ul>



In [41]:
# Grid Search για τον MLP Classifier
mlp = MLPClassifier(max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X, y)
print('Best parameters found:\n', clf.best_params_)

# Παρακάτω παραθέτω το output, μιας και είναι χρονοβόρα διαδικασία:

# GridSearchCV(cv=5, estimator=MLPClassifier(max_iter=100), n_jobs=-1,
#              param_grid={'alpha': [0.0001, 0.05],
#                          'hidden_layer_sizes': [(50, 50, 50), (50, 100, 50),
#                                                 (100,)],
#                          'learning_rate': ['constant', 'adaptive']})

# Best parameters found:
# {'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive'}


Best parameters found:
 {'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive'}


In [42]:

#train - test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=RANDOM_STATE)

print("Gradient Boosting:")
xgb = XGBClassifier(n_estimators=100,use_label_encoder=False, eval_metric='error')
y_pred = xgb.fit(X_train,y_train).predict(X_test)
print("F1-Score: " + str(f1_score(y_test,y_pred,average='weighted')))
print("Accuracy: " + str(balanced_accuracy_score(y_test,y_pred)))

print("SVC:")
svc = SVC(gamma='auto',random_state=RANDOM_STATE)
y_pred = svc.fit(X_train,y_train).predict(X_test)
print("F1-Score: " + str(f1_score(y_test,y_pred,average='weighted')))
print("Accuracy: " + str(balanced_accuracy_score(y_test,y_pred)))


print("MLP:")
mlp = MLPClassifier(random_state=RANDOM_STATE, max_iter=100,alpha=0.0001, hidden_layer_sizes=(100,),learning_rate='adaptive')
y_pred = mlp.fit(X_train,y_train).predict(X_test)
print("F1-Score: " + str(f1_score(y_test,y_pred,average='weighted')))
print("Accuracy: " + str(balanced_accuracy_score(y_test,y_pred)))


print("Bagging:")
bagging = BaggingClassifier(LogisticRegression(random_state=RANDOM_STATE, n_jobs=-1), n_estimators=50, n_jobs=-1, random_state=RANDOM_STATE)
y_pred = bagging.fit(X_train,y_train).predict(X_test)
print("F1-Score: " + str(f1_score(y_test,y_pred,average='weighted')))
print("Accuracy: " + str(balanced_accuracy_score(y_test,y_pred)))

print("Random Forest:")
rfc = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE,n_jobs=-1)
y_pred = rfc.fit(X_train,y_train).predict(X_test)
print("F1-Score: " + str(f1_score(y_test,y_pred,average='weighted')))
print("Accuracy: " + str(balanced_accuracy_score(y_test,y_pred)))
BaggingClassifier(base_estimator=SVC(gamma='auto',random_state=RANDOM_STATE),n_estimators=10, random_state=RANDOM_STATE, max_features=0.7, max_samples=0.7)

print("Random Patches:")
rp = BaggingClassifier(base_estimator=LogisticRegression(random_state=RANDOM_STATE, n_jobs=-1),n_estimators=10, random_state=RANDOM_STATE, max_features=0.7, max_samples=0.7)
y_pred = rp.fit(X_train,y_train).predict(X_test)
print("F1-Score: " + str(f1_score(y_test,y_pred,average='weighted')))
print("Accuracy: " + str(balanced_accuracy_score(y_test,y_pred)))

print("AdaBoost:")
ada = AdaBoostClassifier(base_estimator=LogisticRegression(random_state=RANDOM_STATE,n_jobs=-1) ,n_estimators=10, random_state=RANDOM_STATE)
y_pred = ada.fit(X_train,y_train).predict(X_test)
print("F1-Score: " + str(f1_score(y_test,y_pred,average='weighted')))
print("Accuracy: " + str(balanced_accuracy_score(y_test,y_pred)))

print("LogisticRegression:")
lr = LogisticRegression(random_state=RANDOM_STATE, n_jobs=-1)
y_pred = lr.fit(X_train,y_train).predict(X_test)
print("F1-Score: " + str(f1_score(y_test,y_pred,average='weighted')))
print("Accuracy: " + str(balanced_accuracy_score(y_test,y_pred)))


Gradient Boosting:
F1-Score: 0.8246217930362637
Accuracy: 0.8151808296208805
SVC
F1-Score: 0.8468680264642948
Accuracy: 0.8392989348362883
MLP
F1-Score: 0.851622807347304
Accuracy: 0.8458722231494439
Bagging
F1-Score: 0.8409584742455878
Accuracy: 0.8358229234870953
Random Forest
F1-Score: 0.8066134353605884
Accuracy: 0.7929703046565072
Random Patches
F1-Score: 0.8399049624791819
Accuracy: 0.8333087262970047
AdaBoost
F1-Score: 0.8526366007300631
Accuracy: 0.8483864203395343
LogisticRegression
F1-Score: 0.8329846976316241
Accuracy: 0.8275549592807042


**3.3** Create a classifier that is going to be used in production - in a live system. Use the *test_set_noclass.csv* to make predictions. Store the predictions in a list.  

In [49]:
# BEGIN CODE HERE
cls1 = MLPClassifier(random_state=RANDOM_STATE, max_iter=100,alpha=0.0001, hidden_layer_sizes=(100,),learning_rate='adaptive')
cls2 = AdaBoostClassifier(base_estimator=LogisticRegression(random_state=RANDOM_STATE,n_jobs=-1) ,n_estimators=10, random_state=RANDOM_STATE)
cls3 = SVC(gamma='auto',random_state=RANDOM_STATE)
best_cls = StackingClassifier(estimators=[('MLP',cls1),('AdaBoost',cls2),('SVC',cls3)],final_estimator=LogisticRegression(random_state=RANDOM_STATE,n_jobs=-1),n_jobs=-1)

cls = best_cls.fit(X,y)
#END CODE HERE
test_set = pd.read_csv("test_set_noclass.csv")
predictions = cls.predict(test_set)

<p>Ο τελικός classifier που φτιάξαμε προσπαθήσαμε να είναι αρκετά δημιουργικός μιας και χρησιμοποιεί μία από τις τεχνικές παραγωγής homogenous ensemble όπως το boosting (AdaBoost) ενώ ως base estimator χρησιμοποιούμε γραμμικά μοντέλα (Logistic Regression & SVC) και νευρωνικά μοντέλα (Multi-layer Perceptron classifier). Δηλαδή, υπάρχει μια ποικιλία αλγορίθμων και τεχνικών. Μάλιστα, ο συγκερασμός των επιμέρους classifiers (cls1,cls2,cls3) γίνεται με την τεχνική του stacking έχοντας ως μέτα-μοντέλο έναν Logistic Regression.</p>
<p>Υπολογίσαμε παραπάνω τον Μ.Ο. της μετρικής f1(weighted) και της μετρικής balanced accuracy, οπώς αυτός προέκυψε ύστερα από την διαδικασία του 10 fold CV. Θεωρούμε πως αυτή θα είναι και η ακρίβεια του μοντέλου στις 2 αυτές μετρικές γενικότερα και για αυτό ακριβώς τον λόγο εκπαιδεύσαμε τον τελικό classifier με όλα τα διαθέσιμα data που έχουμε και ο classifier είναι έτοιμος να χρησιμοποιηθεί στο production.</p>

#### This following cell will not be executed. The test_set.csv with the classes will be made available after the deadline and this cell is for testing purposes!!! Do not modify it! ###

In [ ]:
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both should aim above 85%!